In [1]:
import requests

# URL for the 24hr ticker price change statistics endpoint
url = "https://api.binance.com/api/v3/ticker/24hr"

# Make the request to get the ticker information
response = requests.get(url)
ticker_data = response.json()

# Print trading volume for each market
print("24hr Trading Volumes:")
for ticker in ticker_data:
    symbol = ticker['symbol']
    volume = ticker['volume']
    quote_volume = ticker['quoteVolume']
    print(f"Market: {symbol}, Volume: {volume}, Quote Volume: {quote_volume}")


24hr Trading Volumes:
Market: ETHBTC, Volume: 32815.69250000, Quote Volume: 1712.70884578
Market: LTCBTC, Volume: 112250.31400000, Quote Volume: 130.06390823
Market: BNBBTC, Volume: 38935.19900000, Quote Volume: 351.91078675
Market: NEOBTC, Volume: 11842.29000000, Quote Volume: 2.27180611
Market: QTUMETH, Volume: 749.50000000, Quote Volume: 0.63806940
Market: EOSETH, Volume: 33268.70000000, Quote Volume: 6.48445345
Market: SNTETH, Volume: 928085.00000000, Quote Volume: 8.34644284
Market: BNTETH, Volume: 11168.20000000, Quote Volume: 2.24808882
Market: BCCBTC, Volume: 0.00000000, Quote Volume: 0.00000000
Market: GASBTC, Volume: 9385.10000000, Quote Volume: 0.57897705
Market: BNBETH, Volume: 8139.07600000, Quote Volume: 1406.99105480
Market: BTCUSDT, Volume: 33126.31476000, Quote Volume: 2221962249.34297050
Market: ETHUSDT, Volume: 354879.25900000, Quote Volume: 1242880594.57605000
Market: HSRBTC, Volume: 0.00000000, Quote Volume: 0.00000000
Market: OAXETH, Volume: 0.00000000, Quote Volu

In [2]:
ticker_data

[{'symbol': 'ETHBTC',
  'priceChange': '-0.00034000',
  'priceChangePercent': '-0.648',
  'weightedAvgPrice': '0.05219176',
  'prevClosePrice': '0.05249000',
  'lastPrice': '0.05215000',
  'lastQty': '2.25290000',
  'bidPrice': '0.05215000',
  'bidQty': '19.85810000',
  'askPrice': '0.05216000',
  'askQty': '23.85490000',
  'openPrice': '0.05249000',
  'highPrice': '0.05285000',
  'lowPrice': '0.05176000',
  'volume': '32815.69250000',
  'quoteVolume': '1712.70884578',
  'openTime': 1718085979145,
  'closeTime': 1718172379145,
  'firstId': 449494966,
  'lastId': 449668572,
  'count': 173607},
 {'symbol': 'LTCBTC',
  'priceChange': '0.00000300',
  'priceChangePercent': '0.262',
  'weightedAvgPrice': '0.00115870',
  'prevClosePrice': '0.00114500',
  'lastPrice': '0.00114700',
  'lastQty': '3.91900000',
  'bidPrice': '0.00114700',
  'bidQty': '133.03200000',
  'askPrice': '0.00114800',
  'askQty': '168.83600000',
  'openPrice': '0.00114400',
  'highPrice': '0.00118900',
  'lowPrice': '0.0

In [5]:
import requests

# URL for the Binance exchange information endpoint
url = "https://api.binance.com/api/v3/exchangeInfo"

# Make the request to get the exchange info
response = requests.get(url)
exchange_info = response.json()

# Extract unique quote assets
quote_assets = set()
for symbol in exchange_info['symbols']:
    quote_assets.add(symbol['quoteAsset'])

# Print the quote assets
print("Quote Assets:")
for asset in sorted(quote_assets):
    print(asset)


Quote Assets:
AEUR
ARS
AUD
BIDR
BKRW
BNB
BRL
BTC
BUSD
BVND
COP
CZK
DAI
DOGE
DOT
ETH
EUR
FDUSD
GBP
IDRT
JPY
MXN
NGN
PAX
PLN
RON
RUB
TRX
TRY
TUSD
UAH
USDC
USDP
USDS
USDT
UST
VAI
XRP
ZAR


In [7]:
exchange_info['symbols']

[{'symbol': 'ETHBTC',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': 8,
  'quoteAsset': 'BTC',
  'quotePrecision': 8,
  'quoteAssetPrecision': 8,
  'baseCommissionPrecision': 8,
  'quoteCommissionPrecision': 8,
  'orderTypes': ['LIMIT',
   'LIMIT_MAKER',
   'MARKET',
   'STOP_LOSS_LIMIT',
   'TAKE_PROFIT_LIMIT'],
  'icebergAllowed': True,
  'ocoAllowed': True,
  'otoAllowed': True,
  'quoteOrderQtyMarketAllowed': True,
  'allowTrailingStop': True,
  'cancelReplaceAllowed': True,
  'isSpotTradingAllowed': True,
  'isMarginTradingAllowed': True,
  'filters': [{'filterType': 'PRICE_FILTER',
    'minPrice': '0.00001000',
    'maxPrice': '922327.00000000',
    'tickSize': '0.00001000'},
   {'filterType': 'LOT_SIZE',
    'minQty': '0.00010000',
    'maxQty': '100000.00000000',
    'stepSize': '0.00010000'},
   {'filterType': 'ICEBERG_PARTS', 'limit': 10},
   {'filterType': 'MARKET_LOT_SIZE',
    'minQty': '0.00000000',
    'maxQty': '2352.30563333',
    'stepSize': '0.0